In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Corelation Check

In [4]:
df.corr()['Outcome']*100

Pregnancies                  22.189815
Glucose                      46.658140
BloodPressure                 6.506836
SkinThickness                 7.475223
Insulin                      13.054795
BMI                          29.269466
DiabetesPedigreeFunction     17.384407
Age                          23.835598
Outcome                     100.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

## Standard scaling

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=3,test_size=0.2)

# Model

In [48]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [11]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=[X_test,y_test])

Epoch 1/100
20/20 [==============================] - 5s 28ms/step - loss: 0.8172 - accuracy: 0.3518 - val_loss: 0.7388 - val_accuracy: 0.3831
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 0.7252 - accuracy: 0.4283 - val_loss: 0.6756 - val_accuracy: 0.5519
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6592 - accuracy: 0.6319 - val_loss: 0.6334 - val_accuracy: 0.6948
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6119 - accuracy: 0.7345 - val_loss: 0.6007 - val_accuracy: 0.6883
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5747 - accuracy: 0.7606 - val_loss: 0.5771 - val_accuracy: 0.6948
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5481 - accuracy: 0.7785 - val_loss: 0.5603 - val_accuracy: 0.7013
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5277 - accuracy: 0.7736 - val_loss: 0.5498 - val_accuracy: 0.7078
Epoch 8/100


# HyperParemeter Tuning
### 1. How to select appropriate optimizer
### 2. No, of nodes in  a layer
### 3. How to select no. of layers
### 4. All in all one model

In [13]:
# pip install -U keras-tuner

In [14]:
import keras_tuner as kt

# 1

In [15]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
    

In [16]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [17]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=[X_test,y_test])

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.4935 - accuracy: 0.7866 - val_loss: 0.5303 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4827 - accuracy: 0.7834 - val_loss: 0.5244 - val_accuracy: 0.7468
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4744 - accuracy: 0.7850 - val_loss: 0.5212 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4677 - accuracy: 0.7866 - val_loss: 0.5182 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4626 - accuracy: 0.7866 - val_loss: 0.5165 - val_accuracy: 0.7468
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4582 - accuracy: 0.7883 - val_loss: 0.5161 - val_accuracy: 0.7403
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4549 - accuracy: 0.7866 - val_loss: 0.5145 - val_accuracy: 0.7468
Epoch 14

# 2

In [22]:
def build_model(hp):
    model = Sequential()
    
    units = hp.Int('units',min_value=8,max_value=128,step=4)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(units=1,activation='relu'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [23]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [24]:
tuner.search(X_train,y_train,epochs=5,validation_data=[X_test,y_test])

In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 116}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=[X_test,y_test])

Epoch 7/100
20/20 [==============================] - 1s 19ms/step - loss: 0.5045 - accuracy: 0.7834 - val_loss: 1.0359 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5010 - accuracy: 0.7932 - val_loss: 1.0370 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4869 - accuracy: 0.7883 - val_loss: 1.1131 - val_accuracy: 0.7468
Epoch 10/100
20/20 [==============================] - 0s 11ms/step - loss: 0.4996 - accuracy: 0.7980 - val_loss: 1.0282 - val_accuracy: 0.7338
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5030 - accuracy: 0.8013 - val_loss: 1.0505 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4890 - accuracy: 0.7948 - val_loss: 1.0216 - val_accuracy: 0.7078
Epoch 13/100
20/20 [==============================] - 0s 11ms/step - loss: 0.5152 - accuracy: 0.7980 - val_loss: 0.9727 - val_accuracy: 0.7597
Epoch

# 3

In [28]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
        

In [29]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir1',project_name='num_layers')

Reloading Tuner from mydir1\num_layers\tuner0.json


In [30]:
tuner.search(X_train,y_train,epochs=5,validation_data=[X_test,y_test])

In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [33]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=[X_test,y_test])

Epoch 6/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4345 - accuracy: 0.7932 - val_loss: 0.5201 - val_accuracy: 0.7532
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4224 - accuracy: 0.8094 - val_loss: 0.5225 - val_accuracy: 0.7403
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4213 - accuracy: 0.8013 - val_loss: 0.5310 - val_accuracy: 0.7468
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4157 - accuracy: 0.8029 - val_loss: 0.5327 - val_accuracy: 0.7597
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4118 - accuracy: 0.8062 - val_loss: 0.5282 - val_accuracy: 0.7403
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4068 - accuracy: 0.8192 - val_loss: 0.5668 - val_accuracy: 0.7208
Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4038 - accuracy: 0.7997 - val_loss: 0.5320 - val_accuracy: 0.7597
Epoch 13/

# 4

In [49]:
def build_model(hp):
    model = Sequential()
    counter = 0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units' +str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units' +str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1
        
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
            

In [50]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir3',project_name='final')

In [51]:
tuner.search(X_train,y_train,epochs=5,validation_data=[X_test,y_test])

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.5974025726318359
Total elapsed time: 00h 00m 12s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 8,
 'activation0': 'relu',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1}

In [53]:
model = tuner.get_best_models(num_models=1)[0]

In [54]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=[X_test,y_test])

Epoch 6/200
20/20 [==============================] - 2s 17ms/step - loss: 0.6863 - accuracy: 0.6450 - val_loss: 0.6875 - val_accuracy: 0.5974
Epoch 7/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6819 - accuracy: 0.6629 - val_loss: 0.6852 - val_accuracy: 0.5974
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6772 - accuracy: 0.6645 - val_loss: 0.6825 - val_accuracy: 0.5974
Epoch 9/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6731 - accuracy: 0.6612 - val_loss: 0.6799 - val_accuracy: 0.5974
Epoch 10/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6671 - accuracy: 0.6661 - val_loss: 0.6769 - val_accuracy: 0.5974
Epoch 11/200
20/20 [==============================] - 0s 12ms/step - loss: 0.6610 - accuracy: 0.6645 - val_loss: 0.6727 - val_accuracy: 0.5974
Epoch 12/200
20/20 [==============================] - 0s 12ms/step - loss: 0.6537 - accuracy: 0.6645 - val_loss: 0.6668 - val_accuracy: 0.5974
Epoch 1